In [1]:
cat(system('python3 -c "from google.colab import drive\ndrive.mount()"', intern=TRUE), sep='\n', wait=TRUE)

Warning message in system("python3 -c \"from google.colab import drive\ndrive.mount()\"", :
“running command 'python3 -c "from google.colab import drive
drive.mount()"' had status 1”



TRUE


In [2]:
?system

In [3]:
install.packages("rgl", repos = "http://cran.rstudio.com/")
install.packages("ConsRank", repos = "http://cran.rstudio.com/")
library("ConsRank")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘htmlwidgets’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘XML’, ‘rlist’, ‘proxy’, ‘gtools’


Loading required package: rgl

Warning message in rgl.init(initValue, onlyNULL):
“RGL: unable to open X11 display”
Warning message:
“'rgl.init' failed, running with 'rgl.useNULL = TRUE'.”

Attaching package: ‘ConsRank’


The following object is masked from ‘package:base’:

    labels




In [4]:
system("add-apt-repository -y ppa:marutter/rrutter")
system("add-apt-repository -y ppa:marutter/c2d4u")
system("apt-get update")
system("apt install -y r-cran-rstan")

In [6]:
install.packages('ggmap')
install.packages('codetools')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘sp’, ‘RgoogleMaps’, ‘png’, ‘plyr’, ‘rjson’, ‘jpeg’, ‘bitops’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [7]:
library("ggmap")

Loading required package: ggplot2

Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.

Please cite ggmap if you use it! See citation("ggmap") for details.



In [8]:
options(repr.plot.width  = 3,
        repr.plot.height = 3)
Sys.setenv(USE_CXX14 = 1)
library("rstan") # observe startup messages
options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

Loading required package: StanHeaders

rstan (Version 2.21.5, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)



#Notebook Starts Here:

## Studying the Datasets

##First Iris Dataset:

In [232]:
install.packages("stopwords")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [233]:
library('datasets')
data(iris)
X <- iris[1:4]
y <- map_int(iris[['Species']], ~ match(.x, levels(iris[['Species']])))


In [234]:
# scale the data
means <- sapply(X, mean)
sds <- sapply(X, sd)
X <- X %>% map2_df(means, `-`) %>% map2_df(sds, `/`)

In [235]:
data <- list(
  C = max(y),         # num categories
  K = dim(X)[2],      # num features
  N = dim(X)[1],      # num samples
  y = y,              # category of sample n
  x = X,              # sample n
  alpha = c(1, 1, 1), # category prior
  mu_mu = 0,          # feature mean prior
  mu_sigma = 5,       # feature mean prior
  sigma_alpha = 1,    # feature sd prior
  sigma_beta = 1      # feature sd prior
)


In [236]:
train_size <- floor(0.8 * nrow(iris))
ix <- sample(seq_len(nrow(iris)), size = train_size)

X_train <- iris[ix, 1:4]
X_test <- iris[-ix, 1:4]
y_train <- y[ix]
y_test <- y[-ix]


means <- sapply(X_train, mean)
sds <- sapply(X_train, sd)
X_train <- X_train %>% map2_df(means, `-`) %>% map2_df(sds, `/`)
X_test <- X_test %>% map2_df(means, `-`) %>% map2_df(sds, `/`)

data$x <- X_train
data$y <- y_train
data$N <- nrow(X_train)


### Bernoulli Bayes:

Given the Nature of the Data it does not work with Bernoulli Bayes!


### Gaussian Bayes:

In [237]:
fit <- stan(file = '/content/model_gaussian.stan', data = data, iter = 20000)

Warning message in readLines(file, warn = TRUE):
“incomplete final line found on '/content/model_gaussian.stan'”


In [238]:
# retrieve the stan summary object
smry <- summary(fit)$summary

# retrieve our parameters from the summary
mus <- smry[grepl('mu',row.names(smry)),'mean']
sigmas <- smry[grepl('sigma\\[',row.names(smry)),'mean']
thetas <- smry[grepl('theta',row.names(smry)),'mean'] %>% as.vector

# adjust the dimensions so that they are (features, classes)
dim(mus) <- c(data$K, data$C)
dim(sigmas) <- c(data$K, data$C)

In [239]:
# make a prediction by looping over each individual sample
y_pred <- lapply(1:nrow(X_test), \(i) as.matrix(X_test[i,]) |>
  matrix(ncol=4) |>                               # shape the sample so that it broadcasts correctly
  dnorm(mean = mus, sd = sigmas, log = TRUE) |>   # calculate the pd for the various features over all classes
  colSums() |>                                    # sum the pds
  (\(x) x + log(thetas))() |>                     # add class probabilities (sometimes refered to as priors)
  which.max()                                     # predict the class
)

In [240]:
print("The Accuracy Obtained is:")
mean(y_pred==y_test)

[1] "The Accuracy Obtained is:"


[1] 0.9

### Multinomial Bayes:

Dataset is not discrete, and hence multinomial does not apply in this problem

### IMDB Sentiment Analysis:

In [105]:
data <- read.csv(file = '/content/IMDB_Dataset.csv')
library('stopwords')

Impossible to understand how to make a language model work with Stan.

### Raisin Dataset:

In [210]:
data <- read.csv(file = '/content/Raisin_Dataset.csv')
data = data[sample(1:nrow(data)), ]
data[data == "Besni"] <- 2.0
data[data == 'Kecimen'] <- 1.0
data$Class <- as.integer(data$Class)               # Third column is an integer
X = data[1:7]
y = data[8]
X <- X %>% map2_df(means, `-`) %>% map2_df(sds, `/`)
means <- sapply(X, mean)
sds <- sapply(X, sd)
train_size <- floor(0.8 * nrow(data))
ix <- sample(seq_len(nrow(data)), size = train_size)
X_train <- data[ix, 1:7]
X_test <- data[-ix, 1:7]
y_train <- data[ix,8]
y_test <- data[-ix,8]

[1] 900   8
[1] 900   8


In [213]:
data

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
350,67798,418.4081,207.8255,0.8679193,69356,0.6910266,1073.138,1
625,71800,431.0086,218.7158,0.8616806,77521,0.6481548,1187.724,2
755,110785,560.0071,259.3973,0.8862519,116070,0.5585667,1456.757,2
766,121080,573.4036,270.6325,0.8816116,124432,0.7237904,1418.385,2
271,54968,300.9544,234.3896,0.6272465,56851,0.7513395,893.644,1
665,104669,546.6728,248.5279,0.8906859,110984,0.6873457,1398.545,2
366,47602,296.4930,206.2252,0.7184794,49089,0.7211878,840.545,1
147,75314,392.6516,246.0757,0.7792587,77118,0.7047122,1073.768,1
698,93441,396.7908,300.8126,0.6521222,95370,0.7233171,1157.771,2


In [214]:
data2 <- list(
  C = max(y),         # num categories
  K = dim(data)[2]-1,      # num features
  N = dim(data)[1],      # num samples
  y = y,              # category of sample n
  x = X,              # sample n
  alpha = c(1, 1), # category prior
  mu_mu = 0,          # feature mean prior
  mu_sigma = 5,       # feature mean prior
  sigma_alpha = 1,    # feature sd prior
  sigma_beta = 1      # feature sd prior
)


In [215]:
means <- sapply(X_train, mean)
sds <- sapply(X_train, sd)
X_train <- X_train %>% map2_df(means, `-`) %>% map2_df(sds, `/`)
X_test <- X_test %>% map2_df(means, `-`) %>% map2_df(sds, `/`)

In [217]:
data2$x <- X_train
data2$y <- y_train
data2$N <- nrow(X_train)


### Gaussian Model:

In [218]:
fit <- stan(file = '/content/model_gaussian.stan', data = data2, iter = 1000)

Warning message in readLines(file, warn = TRUE):
“incomplete final line found on '/content/model_gaussian.stan'”


In [219]:
# retrieve the stan summary object
smry <- summary(fit)$summary

# retrieve our parameters from the summary
mus <- smry[grepl('mu',row.names(smry)),'mean']
sigmas <- smry[grepl('sigma\\[',row.names(smry)),'mean']
thetas <- smry[grepl('theta',row.names(smry)),'mean'] %>% as.vector

# adjust the dimensions so that they are (features, classes)
dim(mus) <- c(data$K, data$C)
dim(sigmas) <- c(data$K, data$C)

In [223]:
# make a prediction by looping over each individual sample
y_pred <- lapply(1:nrow(X_test), \(i) as.matrix(X_test[i,]) |>
  matrix(ncol=7) |>                               # shape the sample so that it broadcasts correctly
  dnorm(mean = mus, sd = sigmas, log = TRUE) |>   # calculate the pd for the various features over all classes
  colSums() |>                                    # sum the pds
  (\(x) x + log(thetas))() |>                     # add class probabilities (sometimes refered to as priors)
  which.max()                                     # predict the class
)

ERROR: ignored

In [ ]:
mean(y_pred==y_test)

In [231]:
X_test[1,]

Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-0.3905625,0.01521064,-0.688234,0.8842502,-0.3160537,-0.8953966,0.09362229
